In [ ]:
! pip install tsai
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 9.7 MB/s 
     |████████████████████████████████| 2.5 MB 54.0 MB/s 
Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 50.1MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  
Looking in ind

In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import json
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))
    
    
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append([seq_x])
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
# define scope of configs
AHB_list = [10]
n_steps = [1]
mul_uni = ['uni','mul']
n_layers = [2, 3, 4]
dropouts = [0.1, 0.2, 0.4]
fc_dropouts = [0.0, 0.2, 0.4]
d_models = [128, 256, 512]
d_ffs = [256, 512, 1024]
epoches = [10, 25, 50, 100]

count = 1
performance_json = []
for g in AHB_list:
    for i in n_steps:
        for j in n_layers:
            for c in dropouts:
                  for d in fc_dropouts:
                        for e in d_models:
                              for h in d_ffs:
                                  for f in mul_uni:
                                      for k in epoches:
                                          print('PROGRESS: [ ' + str(count) + ' / ' + str(len(AHB_list)*len(n_steps)*len(mul_uni)*len(n_layers)*len(dropouts) \
                                          *len(fc_dropouts)*len(d_models)*len(d_ffs)*len(epoches)) + ' ]')
                                          if f == 'uni':
                                              if g == 13:
                                                  raw_seq = list(df_list[g]['Country_Level'])
                                                  ahb = 'Country_Level'
                                              else:    
                                                  raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                                  ahb = 'AHB_'+str(g+1)
                                              n_features = 1
                                              scaler = get_scaler('minmax')
                                              raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1)).reshape((len(raw_seq)))
                                              X, y = uni_split_sequence(raw_seq_arr, i)
                                          if f == 'mul':
                                              if g == 13:
                                                  raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                                  ahb = 'Country_Level'
                                                  scaler_y = get_scaler('minmax')
                                                  scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                              else:    
                                                  raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                                  ahb = 'AHB_'+str(g+1)
                                                  scaler_y = get_scaler('minmax')
                                                  scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                              n_features = len(raw_seq[0])-1
                                              scaler = get_scaler('minmax')
                                              raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                              X, y = multi_split_sequences(raw_seq_arr, i)
                                              
                                          from tsai.all import *
                                          splits = TimeSplitter(int(len(y)*0.1))(y) 
                                          batch_tfms = TSStandardize()
                                          fcst = TSForecaster(X, y, splits=splits, path='models', batch_tfms=batch_tfms, bs=16, arch=TST, metrics=mse, 
                                                              arch_config={'n_layers':j, 'dropout':c, 'fc_dropout':d, 'd_model':e, 'd_ff':h})
                                          fcst.fit_one_cycle(k, 1e-3)
                                          fcst.export("fcst.pkl")

                                          from tsai.inference import load_learner
                                          fcst = load_learner("models/fcst.pkl", cpu=False)
                                          raw_preds, target, preds = fcst.get_X_preds(X[splits[1]], y[splits[1]])
                                          print(raw_preds.shape)
                                          mae = mean_absolute_error(target, preds)
                                          mse = mean_squared_error(target, preds)
                                          rmse = mean_squared_error(target, preds, squared = False)
                                          mape = mean_absolute_percentage_error(target, preds)


                                          if f == 'uni':
                                              preds_inverse = scaler.inverse_transform(preds)
                                              target_inverse = scaler.inverse_transform(target.reshape((len(target),-1)))
                                          if f == 'mul':
                                              preds_inverse = scaler_y.inverse_transform(preds)
                                              target_inverse = scaler_y.inverse_transform(target.reshape((len(target),-1)))

                                          mae_inverse = mean_absolute_error(target_inverse, preds_inverse)
                                          mse_inverse = mean_squared_error(target_inverse, preds_inverse)
                                          rmse_inverse = mean_squared_error(target_inverse, preds_inverse, squared = False)
                                          mape_inverse = mean_absolute_percentage_error(target_inverse, preds_inverse)
                                          
                                          performance_json.append({'model': 'TST', 'AHB_number': ahb, 'Multi_Uni': f, 'epoch': k, 
                                                      'n_layers':j, 'dropout':c, 'fc_dropout':d, 'd_model':e, 'd_ff':h, 'n_steps': i, 'MAE_normalize': float(mae), 
                                                      'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), 
                                                      'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})

                                          json_formatted_str = json.dumps(performance_json[-1], indent=2, cls=NumpyEncoder)
                                          print(json_formatted_str)
                                          import psutil 
                                          print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                          print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                          from datetime import datetime, timezone, timedelta
                                          timezone_offset = 7.0  # Bangkok Time (UTC+07:00) 
                                          tzinfo = timezone(timedelta(hours=timezone_offset))
                                          print(datetime.now(tzinfo))
                                          
                                          count = count + 1
                                          

In [ ]:
name = "all_performance_TST_1nsteps_AHB11"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_TST_AHB1.json /content/drive/MyDrive/Depression
! cp /content/all_performance_TST_1nsteps_AHB11.csv /content/drive/MyDrive/Depression